# Calculate the $\Delta G$ between two residues


Let A and B are the two residues in a chemical reaction then,

$AH + B ⇌ A^{-} + BH$
 
Free energy
 
$\Delta G = - RT \ln\frac{[A^{-}] [BH]} {[AH]  [B]}$
 
But for microstate since A and B are need to appear in each microstate then
 

 
 $\Delta G = - RT \ln\frac{[A^{-}| BH ]} {[AH | B]}$
 
where RT =  1.36 in log10 and  RT = 0.593  in ln 





In [1]:
import ms_analysis as msa

In [2]:
mc = msa.MSout("../ms_out/pH5eH0ms.txt")

In [3]:
ms_orig_lst = [[ms.E, ms.count, ms.state] for  ms in list((mc.microstates.values()))]
ms_orig_lst = sorted(ms_orig_lst, key = lambda x:x[0])

In [5]:
# res1 = ["ASPA48", -1]
# res2 = ["GLUA35", 0]
# find the deltaG energies. This will take the 
# ms_list is in the form of ms_orig_lst
def Count_residue_condition(res1, res2, ms_list):
    # first find the residues conformer list.
    res1_iconf = {conf.iconf: int(conf.crg) for conf in msa.conformers \
                  if (conf.resid[:4] + str(int(conf.resid[4:8]))) in res1[0]}

    res2_iconf = {conf.iconf: int(conf.crg) for conf in msa.conformers \
                  if (conf.resid[:4] + str(int(conf.resid[4:8]))) in res2[0]}
    
    combine_iconf = {**res1_iconf, **res2_iconf}
    # this is for look up in order to track whether given residues are in fixed ms residues list or not.
    look_up = set()
    #print(mc.fixed_iconfs)
    for i, j in combine_iconf.items():
        if i in mc.fixed_iconfs:
            look_up.add(i)
    if look_up:
        for x in look_up:
            if x in res1_iconf.keys():
                print(f"{res1[0]} is fixed with charge {res1_iconf[x]}.Try free residue.")
            if x in res2_iconf.keys():
                print(f"{res2[0]} is fixed with charge {res2_iconf[x]}. Try free residue.")
        return("You have fixed residue in ms list. Try free residues")

    #now find the confomers position in ms state in order to speed up searching position. 
    #Take first ms state for this.
    first_ms_lookup = list(mc.microstates.values())[0].state

    for i in range(len(first_ms_lookup)):
        if first_ms_lookup[i] in res1_iconf:
            res1_position_ms = i
            #print(res1_position_ms)
        if first_ms_lookup[i] in res2_iconf:
            res2_position_ms = i
            #print(res2_position_ms)
    # group reisudes based on given condition. Make list of the conformers for look up.
    res1_iconf_given_crg = [x for x, y in res1_iconf.items() if y == res1[1]]
    res2_iconf_given_crg = [x for x, y in res2_iconf.items() if y == res2[1]]
    # count based given charge condition
    count_given_condition = 0
    for i in ms_list:
        if i[2][res1_position_ms] in res1_iconf_given_crg and i[2][res2_position_ms] in res2_iconf_given_crg:
            count_given_condition += i[1]
    return count_given_condition 


In [6]:
import math
# RT =  1.36 #1.36 if log10 and if RT = 0.593 if in ln 
def deltaG_from_count(reactant, product):
    # if residues are fixed then left or right will not return int
    if type(reactant) != int or type(product) != int:
        return ("Residues choosen are fixed")
    elif reactant == 0 or  product == 0:
        return ("Value Error")
    else:
        deltaG = round(- 1.36 * math.log10(product /reactant), 3)
        return deltaG
   

In [8]:
# Glu-35 has experimental pKa 6.1 whereas Asp-52 has 3.4
# Glu-35 is proton donor and ASP52 is acceptor.
# reaction: GLU35(0) + ASP52(-1) == GLU35(-1) + ASP52(0)
# chisquare: GLU35 = 0.573, ASP52 = 0.094
# slope: GLU35 = 0.954, ASP52 = 0.940
reactant = Count_residue_condition(["GLUA35", 0], ["ASPA52", -1], ms_orig_lst)
product=  Count_residue_condition(["GLUA35", -1], ["ASPA52", 0],  ms_orig_lst)
print(f"Reactant number: {reactant}")
print(f"Product number: {product}")
print(f"Total reactant and product: {reactant + product}")
print(f"Total all count: {mc.N_ms}")
print(f"DeltaG: {deltaG_from_count(reactant, product)} Kcal/Mol")


Reactant number: 817457
Product number: 24832
Total reactant and product: 842289
Total all count: 1500000
DeltaG: 2.064 Kcal/Mol
